# 🧠 Classificação de Imagens com Redes Neurais Convolucionais

## 📋 Objetivo
Nesta atividade prática, vamos desenvolver uma Rede Neural Convolucional para classificar imagens do dataset STL-10. Inicialmente, utilizaremos a arquitetura LeNet-5 e, em seguida, modificaremos a arquitetura para melhorar o desempenho, explorando técnicas como Dropout, regularização e data augmentation.

## 📊 Especificações Técnicas

1. **Arquitetura LeNet-5**  
   Implementar a arquitetura LeNet-5, que consiste em 2 camadas convolucionais seguidas de 2 camadas totalmente conectadas. 

2. **Modificações na Arquitetura**  
   Testar diferentes configurações de rede, incluindo a EfficientNet, para melhorar o desempenho. Adicionar/remover camadas, modificar otimizador e parâmetros.
   Implementar técnicas de regularização como Dropout e L2.

3. **Data Augmentation**  
   Aplicar técnicas de data augmentation para aumentar a diversidade do conjunto de dados de treinamento, como rotação, translação e espelhamento.

4. **Treinamento e Avaliação**
    Treinar a rede neural com o conjunto de dados STL-10 e avaliar o desempenho utilizando métricas como acurácia, precisão e recall. Plotar as curvas de loss e acurácia para o conjunto de treinamento e teste.

5. **Visualização de Resultados**  
   Visualizar as previsões da rede neural em algumas imagens do conjunto de teste, destacando as classes previstas e reais.
   Utilizar técnicas de visualização como Grad-CAM para entender quais regiões da imagem influenciam a decisão da rede.

## 🗃️ Base de Dados
Este projeto utiliza o dataset "STL-10 Image Recognition" do Kaggle, que contém características extraídas de imagens de 96x96 pixels. O dataset é dividido em 10 classes: avião, pássaro, carro, gato, veado, cachorro, cavalo, macaco, navio e caminhão. O conjunto de dados contém 500 imagens de treinamento por classe (10 classes) e 800 imagens de teste por classe. Além disso, há 100.000 imagens não rotuladas para aprendizado não supervisionado.

[📁STL-10 Image Recognition Dataset](https://www.kaggle.com/datasets/jessicali9530/stl10)

## 📚 Material de Referência

[🔗 Apresentação de Redes Neurais Convolucionais](https://docs.google.com/presentation/d/1LHjSGZ9YAl0u6aZ-mKuIocX02Fpd3WNVq_Dbzh2-wDY/edit)

[🔗 LeNet-5 Paper](./Lecun98.pdf)

[🔗 LeNet Architecture: A Complete Guide](https://www.kaggle.com/code/blurredmachine/lenet-architecture-a-complete-guide)

[🔗 Hands on guide to LeNet-5 (The Complete Info)](https://syedabis98.medium.com/hands-on-guide-to-lenet-5-the-complete-info-b2ae631db34b)

## 📦 Importação de Bibliotecas

As bibliotecas necessárias para o desenvolvimento do modelo são importadas abaixo.

In [206]:
# Bibliotecas para manipulação de arquivos e dados
import os             # Operações com o sistema de arquivos
import cv2            # Processamento de imagens
import json           # Manipulação de arquivos JSON
import requests       # Requisições HTTP para download de arquivos
import numpy as np    # Operações numéricas e manipulação de arrays
import pandas as pd   # Manipulação e análise de dados tabulares
import seaborn as sns # Visualização de dados estatísticos

# Bibliotecas para visualização de dados
import matplotlib.pyplot as plt  # Criação de gráficos e visualizações

# Bibliotecas para construção e treinamento de modelos de deep learning
from tensorflow.keras import regularizers  # Regularização para evitar overfitting
from tensorflow.keras.models import Sequential, Model  # Modelos sequencial e funcional do Keras
from tensorflow.keras.layers import (Dense, Conv2D, MaxPooling2D, AvgPool2D, 
                                         Flatten, Dropout, Input, GlobalAveragePooling2D, 
                                         BatchNormalization, Activation)  # Camadas para redes neurais
from tensorflow.keras.applications import EfficientNetB0  # Arquitetura EfficientNet pré-definida
from tensorflow.keras.applications.efficientnet import preprocess_input  # Pré-processamento para EfficientNet
from tensorflow.keras.regularizers import l2          # Regularização L2
from tensorflow.keras.optimizers import Adam          # Otimizador Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping  # Callbacks para treinamento
import tensorflow as tf                                # Biblioteca principal do TensorFlow
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Para data augmentation
from tensorflow.keras.utils import to_categorical # Conversão de rótulos para one-hot encoding

# Bibliotecas para manipulação de dados e avaliação de modelos
from sklearn.model_selection import train_test_split  # Divisão do dataset em treino e teste
from sklearn.metrics import recall_score, precision_score, f1_score, confusion_matrix  # Métricas de avaliação

# Biblioteca para integração com o Kaggle
import kagglehub  # Download de datasets do Kaggle

# Biblioteca para manipulação de tempo
import time  # Medição de tempo de execução


## 📥 Carregamento dos Dados

Para garantir a reprodutibilidade do experimento, vamos baixar os dados diretamente do Kaggle.

In [207]:
def download_dataset(dataset_name: str) -> str:
    """
    Baixa o datasetdo Kaggle e salva os arquivos JSON de treino e teste.

    Parameters:
    ----------
        dataset_name: str
            Nome do dataset no Kaggle. Deve estar no formato "username/dataset-name".
            Exemplo: "jessicali9530/stl10".

    Returns:
    ----------
        str
            Caminho onde os arquivos JSON foram salvos.
    """
    path = kagglehub.dataset_download(dataset_name)
    
    # URLs dos arquivos JSON
    train_url = "https://storage.googleapis.com/kaggle-forum-message-attachments/2852410/20764/train.json"
    test_url = "https://storage.googleapis.com/kaggle-forum-message-attachments/2852411/20765/test.json"
    
    # Caminhos para salvar os arquivos
    train_path = os.path.join(path, "train.json")
    test_path = os.path.join(path, "test.json")
    
    # Download do arquivo de treino
    with requests.get(train_url, stream=True) as r:
        r.raise_for_status()
        with open(train_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    
    # Download do arquivo de teste
    with requests.get(test_url, stream=True) as r:
        r.raise_for_status()
        with open(test_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    
    return path

## 🔄 Processamento de Dados

As funções a seguir permitem o carregamento, separação e preparação dos dados para o treinamento do modelo.

In [208]:
# Funções para carregar e pré-processar os dados
def load_train_test_labels(path: str) -> tuple:
    """
    Carrega os rótulos de treino e teste a partir dos arquivos JSON.

    Parametrs:
    ----------
    path : str
        Caminho do diretório onde os arquivos JSON estão localizados.

    Returns:
    -------
    tuple
        Uma tupla contendo os rótulos de treino e teste.
    """
    with open(os.path.join(path, 'train.json'), 'r') as f:
        train_labels = json.load(f)
    with open(os.path.join(path, 'test.json'), 'r') as f:
        test_labels = json.load(f)
    return train_labels, test_labels

In [209]:
def load_single_image(path: str) -> np.ndarray:
  """
    Carrega uma única imagem a partir de um caminho especificado.

    Parameters:
    ----------
        path: str
            Caminho do arquivo da imagem.

    Returns:
    ----------
        np.ndarray
            A imagem carregada como um array NumPy.
            Retorna None se a imagem não puder ser carregada.
  
  """
  img = cv2.imread(path)
  if img is None:
    print(f"ERRO: Não foi possível carregar a imagem: {path}")
    return None
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

In [210]:
def load_images_from_folder(folder: str, labels_json: list, folder_label: str) -> tuple:
    """
    Carrega as imagens de um diretório e associa os rótulos a elas.

    Parameters:
    ----------
        folder: str
            Caminho do diretório onde as imagens estão localizadas.
        labels_json: list
            Rótulos das imagens em formato JSON.
        folder_label: str
            Nome da pasta que contém as imagens.

    Returns:
    ----------
        tuple
            Uma tupla contendo as imagens e os rótulos correspondentes.
    """
    images, labels = [], []
    
    # Verificar se o diretório existe
    if not os.path.isdir(folder):
        print(f"ERRO: Diretório não encontrado: {folder}")
        print(f"Diretórios disponíveis: {os.listdir(os.path.dirname(folder) if os.path.dirname(folder) else '.')}")
        return np.array([]), np.array([])
    
    # Listar conteúdo do diretório
    print(f"Conteúdo do diretório {folder}: {os.listdir(folder)}")
    
    # Verificar estrutura do dataset no Kaggle
    print(f"Procurando por imagens no caminho: {os.path.join(folder, folder_label)}")
    train_dir = os.path.join(folder, folder_label)
    if os.path.isdir(train_dir):
        folder = train_dir  # Atualiza o caminho se as imagens estiverem nesse subdiretório
        print(f"Usando diretório de imagens: {folder}")
    
    # Verificar como os rótulos estão estruturados
    if labels_json and isinstance(labels_json, list) and len(labels_json) > 0:
        print(f"Exemplo de estrutura de rótulo: {labels_json[0]}")
    
    # Tentar carregar imagens com base na estrutura do dataset
    filenames = os.listdir(folder) if os.path.isdir(folder) else []
    for filename in filenames:
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            # print(f"Carregando imagem: {filename}")
            img_path = os.path.join(folder, filename)
            img = load_single_image(img_path)
            if img is not None:
                label = next((item['label'] for item in labels_json if item['file'] == f'{folder_label}/{filename}'), None)
                if label is not None:
                    labels.append(label)
                    images.append(img)
                else:
                    print(f"Não foi possível encontrar o rótulo para a imagem: {filename}")
    
    if not images:
        print("ALERTA: Nenhuma imagem foi carregada!")
    else:
        print(f"Total de imagens carregadas: {len(images)}")
    
    return np.array(images), np.array(labels)

In [211]:
def one_hot_encode(labels: np.ndarray, num_classes: int) -> np.ndarray:
    """
    Codifica os rótulos em formato one-hot.

    Parameters:
    ----------
        labels: np.ndarray
            Rótulos a serem codificados.
        num_classes: int
            Número de classes para a codificação one-hot.

    Returns:
    ----------
        np.ndarray
            Rótulos codificados em formato one-hot.
    """
    return to_categorical(labels.astype(np.int32), num_classes=num_classes)

In [212]:
def one_hot_decode(one_hot_labels: np.ndarray) -> np.ndarray:
    """
    Decodifica os rótulos one-hot de volta para os rótulos originais.

    Parameters:
    ----------
        one_hot_labels: np.ndarray
            Rótulos codificados em formato one-hot.

    Returns:
    ----------
        np.ndarray
            Rótulos decodificados.
    """
    return np.argmax(one_hot_labels, axis=1)

In [213]:
def dataset_split(X: np.ndarray, y: np.ndarray) -> tuple:
    """
    Divide o dataset em conjuntos de treino e teste
    A divisão é feita em 60% para treino, 20% para teste e 20% para validação.

    Parameters:
    ----------
        X: np.ndarray
            Imagens do dataset.
        y: np.ndarray
            Rótulos do dataset.

    Returns:
    ----------
        tuple
            Conjuntos de treino, teste e validação (X_train, X_test, y_train, y_test).
    """
    # Divisão do dataset em treino, teste e validação
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
    return X_train.astype(np.float32), X_test.astype(np.float32), y_train.astype(np.int32), y_test.astype(np.int32)

## 🏗️ Arquitetura LeNet-5

Implementação da arquitetura LeNet-5 para classificação de imagens.

In [214]:
# Implementação da arquitetura LeNet-5 melhorada
def improved_lenet_5(input_shape: tuple=(128, 128, 3), num_classes: int=10) -> Sequential:
    """
    Cria uma versão melhorada do modelo LeNet-5 para classificação de imagens.

    Parameters:
    ----------
        input_shape (tuple): Forma de entrada das imagens.
        num_classes (int): Número de classes para classificação.

    Returns:
    ----------
        model (Sequential): Modelo LeNet-5 melhorado compilado.
    """    
    model = Sequential()
    
    # Primeiro bloco convolucional - camada inicial
    model.add(Conv2D(32, kernel_size=(3, 3), padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(32, kernel_size=(3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    # Segundo bloco convolucional - aumentando profundidade
    model.add(Conv2D(64, kernel_size=(3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(64, kernel_size=(3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    # Terceiro bloco convolucional - aumentando profundidade
    model.add(Conv2D(128, kernel_size=(3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(128, kernel_size=(3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    # Camada totalmente conectada - classificação
    model.add(Flatten())
    model.add(Dense(512, kernel_regularizer=regularizers.l2(0.001)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(256, kernel_regularizer=regularizers.l2(0.001)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(num_classes, activation='softmax'))
    
    # Compilando com otimizador Adam melhorado
    optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
    model.compile(
        loss='categorical_crossentropy', 
        optimizer=optimizer, 
        metrics=['accuracy']
    )
    return model

In [215]:
def lenet_5(input_shape: tuple=(128, 128, 3), num_classes: int=10) -> Sequential:
    """
    Cria o modelo LeNet-5 para classificação de imagens.

    Parameters:
    ----------
        input_shape (tuple): Forma de entrada das imagens.
        num_classes (int): Número de classes para classificação.

    Returns:
    ----------
        model (Sequential): Modelo LeNet-5 compilado.
    """    
    model = Sequential()
    model.add(Conv2D(6, kernel_size=(5, 5), activation='tanh', input_shape=input_shape))
    model.add(AvgPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(16, kernel_size=(5, 5), activation='tanh'))
    model.add(AvgPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(120, kernel_size=(5, 5), activation='tanh'))
    model.add(Flatten())
    model.add(Dense(84, activation='tanh'))
    model.add(Dropout(0.5))

    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

## 💪 Treinamento

Treinamento do modelo utilizando data augmentation e técnicas de regularização.

In [216]:
def model_fit(model: Sequential, X_train: np.ndarray, y_train: np.ndarray, X_test: np.ndarray, y_test: np.ndarray, epochs: int = 50, batch_size: int =32):
    """
    Treina o modelo com os dados usando data augmentation.

    Parameters:
    ----------
        model: Sequential
            O modelo a ser treinado.
        X_train: np.ndarray
            Conjunto de dados de treino.
        y_train: np.ndarray
            Rótulos de treino.
        X_test: np.ndarray
            Conjunto de dados de teste.
        y_test: np.ndarray
            Rótulos de teste.
        epochs: int
            Número de épocas para o treinamento.
        batch_size: int
            Tamanho do lote para o treinamento.

    Returns:
    ----------
        model: Sequential
            O modelo treinado.
    """
    # Enhanced Data Augmentation
    train_datagen = ImageDataGenerator(
        rotation_range=20,          # Increased rotation range
        width_shift_range=0.2,      # Increased shift range
        height_shift_range=0.2,     # Increased shift range
        shear_range=0.15,           # Added shear transformation
        zoom_range=0.15,            # Added zoom capability
        horizontal_flip=True,       # Horizontal flips
        vertical_flip=False,        # Generally not useful for object recognition
        brightness_range=[0.8, 1.2],# Brightness variation
        fill_mode='nearest',        # Fill strategy for created pixels
        rescale=1./255             # Normalize pixel values
    )

    # For the test set, only normalization
    test_datagen = ImageDataGenerator(rescale=1./255)
    
    # Create generators
    train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
    test_generator = test_datagen.flow(X_test, y_test, batch_size=batch_size, shuffle=False)
    
    # Define callbacks for improved training
    callbacks = [
        # Early stopping to prevent overfitting
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        ),
        # Learning rate reduction when plateauing
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-6
        ),
        # Model checkpoint to save the best model
        tf.keras.callbacks.ModelCheckpoint(
            'best_model.h5',
            save_best_only=True,
            monitor='val_accuracy',
            mode='max'
        )
    ]
    
    # Train with callbacks
    history = model.fit(
        train_generator,
        epochs=epochs,
        validation_data=test_generator,
        callbacks=callbacks
    )
    
    return history

In [217]:
def model_fit_no_augmentation(model: Sequential, X_train: np.ndarray, y_train: np.ndarray, X_test: np.ndarray, y_test: np.ndarray, epochs: int = 50, batch_size: int =32):
    """
    Treina o modelo sem data augmentation.

    Parameters:
    ----------
        model: Sequential
            O modelo a ser treinado.
        X_train: np.ndarray
            Conjunto de dados de treino.
        y_train: np.ndarray
            Rótulos de treino.
        X_test: np.ndarray
            Conjunto de dados de teste.
        y_test: np.ndarray
            Rótulos de teste.
        epochs: int
            Número de épocas para o treinamento.
        batch_size: int
            Tamanho do lote para o treinamento.

    Returns:
    ----------
        model: Sequential
            O modelo treinado.
    """
    # Normalização dos dados (sem data augmentation)
    # X_train = X_train.astype('float32') / 255.0
    # X_test = X_test.astype('float32') / 255.0
    return model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))

## 📈 Avaliação do Modelo

Avaliação do modelo utilizando o conjunto de dados de teste e plotagem das curvas de loss e acurácia.

In [218]:
def evaluate_model(model: Sequential, X_test: np.ndarray, y_test: np.ndarray) -> tuple:
    """
    Avalia o modelo com os dados de teste.

    Parameters:
    ----------
        model: Sequential
            O modelo a ser avaliado.
        X_test: np.ndarray
            Conjunto de dados de teste.
        y_test: np.ndarray
            Rótulos de teste.

    Returns:
    ----------
        tuple
            Acurácia e perda do modelo nos dados de teste.
    """
    scores = model.evaluate(X_test, y_test, return_dict=True)
    print(scores)
    print(f"Test loss: {scores['loss']:.4f}")
    print(f"Test accuracy: {scores['accuracy']:.4f}")
    return scores

## 🔄 Modificação da Arquitetura

Implementação de uma arquitetura melhorada com EfficientNet e técnicas como Dropout, regularização e data augmentation.

In [ ]:
# # Definição da camada de data augmentation
# data_augmentation = Sequential([
#     tf.keras.layers.RandomFlip('horizontal'),
#     tf.keras.layers.RandomRotation(0.1),
#     tf.keras.layers.RandomZoom(0.1),
#     tf.keras.layers.RandomContrast(0.1),
#     tf.keras.layers.RandomBrightness(0.1),
#     tf.keras.layers.RandomTranslation(0.1, 0.1),
#     tf.keras.layers.RandomHeight(0.1),
#     tf.keras.layers.RandomWidth(0.1),
#     tf.keras.layers.RandomCrop(0.1, 0.1),
# ], name='data_augmentation')

def create_improved_efficientnet(input_shape=(128, 128, 3), num_classes=10):
    """
    Cria um modelo EfficientNet melhorado com transfer learning.
    - Inclui data augmentation e pré-processamento.
    - Utiliza regularização L2 nas camadas densas.
    - Implementa uma arquitetura modular utilizando a API funcional.
    
    Parâmetros:
      input_shape: formato das imagens de entrada.
      num_classes: número de classes para classificação.
      
    Retorna:
      Modelo Keras compilado.
    """
    # Entrada do modelo
    inputs = Input(shape=input_shape)
    
    # Aplicação da data augmentation
    x = tf.keras.layers.RandomFlip('horizontal')(inputs)
    x = tf.keras.layers.RandomRotation(0.1)(x)
    x = tf.keras.layers.RandomZoom(0.1)(x)
    x = tf.keras.layers.RandomContrast(0.1)(x)
    # x = tf.keras.layers.RandomBrightness(0.1)(x)
    # x = tf.keras.layers.RandomTranslation(0.1, 0.1)(x)
    # x = tf.keras.layers.RandomHeight(0.1)(x)
    # x = tf.keras.layers.RandomWidth(0.1)(x)
    
    # Pré-processamento específico para EfficientNet
    x = preprocess_input(x)
    
    # Carregar o modelo base com pesos do ImageNet e sem a camada superior
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape, name='efficientnetb0')
    base_model.trainable = False  # Congela o base model inicialmente
    
    # Passa os dados pelo base model
    x = base_model(x, training=False)
    
    # Cabeça de classificação
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dense(512, kernel_regularizer=regularizers.l2(1e-4))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs, outputs, name='improved_efficientnet')
    
    # Compilação do modelo
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

def fine_tune_efficientnet(model, train_data, validation_data, epochs_initial=20, epochs_fine_tune=30):
    """
    Realiza um treinamento em duas etapas:
      1. Treinamento inicial apenas das camadas superiores (com o base model congelado).
      2. Fine-tuning: descongela as últimas 30 camadas do base model e retreina com taxa de aprendizado reduzida.
      
    Adiciona callbacks de ModelCheckpoint, ReduceLROnPlateau e EarlyStopping.
    
    Parâmetros:
      model: modelo EfficientNet compilado.
      train_data: gerador ou conjunto de dados de treinamento.
      validation_data: gerador ou conjunto de dados de validação.
      epochs_initial: número de épocas para treinamento inicial.
      epochs_fine_tune: número de épocas para fine-tuning.
      
    Retorna:
      Modelo treinado e uma lista com os históricos de treinamento (fase inicial e fine-tuning).
    """
    # Callbacks para a fase inicial
    checkpoint_initial = ModelCheckpoint('efficientnet_initial.h5', save_best_only=True, 
                                           monitor='val_accuracy', verbose=1)
    reduce_lr_initial = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, 
                                          min_lr=1e-6, verbose=1)
    early_stop_initial = EarlyStopping(monitor='val_loss', patience=5, 
                                       restore_best_weights=True, verbose=1)
    
    # Etapa 1: Treinamento das camadas superiores
    
    steps_per_epoch = len(train_data)

    validation_steps = len(validation_data)
    
    history_initial = model.fit(
        train_data,
        validation_data=validation_data,
        epochs=epochs_initial,
        callables=[checkpoint_initial, reduce_lr_initial, early_stop_initial]
        # steps_per_epoch=steps_per_epoch,
        # validation_steps=validation_steps
    )
    
    # Etapa 2: Fine-tuning
    # Identifica o base model pelo nome (garantindo acesso correto mesmo com a API funcional)
    base_model = model.get_layer('efficientnetb0')
    
    # Descongela as últimas 30 camadas do base model
    for layer in base_model.layers[-30:]:
        layer.trainable = True
    
    # Recompila com uma taxa de aprendizado menor para fine-tuning
    optimizer = Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    # Callbacks para a fase de fine-tuning
    checkpoint_ft = ModelCheckpoint('efficientnet_fine_tuned.h5', save_best_only=True, 
                                      monitor='val_accuracy', verbose=1)
    reduce_lr_ft = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, 
                                     min_lr=1e-7, verbose=1)
    early_stop_ft = EarlyStopping(monitor='val_loss', patience=5, 
                                  restore_best_weights=True, verbose=1)
    
    history_fine_tune = model.fit(
        train_data,
        validation_data=validation_data,
        epochs=epochs_fine_tune,
        callbacks=[checkpoint_ft, reduce_lr_ft, early_stop_ft]
    )
    
    return model, [history_initial, history_fine_tune]

## 📊 Avaliação do Modelo

Gráficos de loss, acurácia, precisão, recall, F1-score, curva roc e matriz de confusão para o modelo melhorado. Visualização das previsões da rede neural em algumas imagens do conjunto de teste, destacando as classes previstas e reais. Utilização de técnicas de visualização como Grad-CAM para entender quais regiões da imagem influenciam a decisão da rede.

In [220]:
# Funções para visualização de resultados
def imprime_loss_accuracy(history: tf.keras.callbacks.History):
    """
    Plota a perda e a acurácia do modelo durante o treinamento.
    Parameters:
    ----------
        history: tf.keras.callbacks.History
            Histórico do treinamento do modelo.

    Returns:
    ----------
        None
    """
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

def imprime_recall_precision_f1_score(model: Sequential, X_test: np.ndarray, y_test: np.ndarray):
    """
    Calcula e imprime as métricas de recall, precisão e F1-score do modelo.
    Parameters:
    ----------
        model: Sequential
            O modelo a ser avaliado.
        X_test: np.ndarray
            Conjunto de dados de teste.
        y_test: np.ndarray
            Rótulos de teste.
    Returns:
    ----------
        recall: float
            Valor do recall.
        precision: float
            Valor da precisão.
        f1: float
            Valor do F1-score.
    """
    y_pred = model.predict(X_test)
    y_pred = (y_pred > 0.5)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print(f'Recall: {recall:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'F1 Score: {f1:.4f}')
    return recall, precision, f1

def imprime_matriz_confusao(y_test: np.ndarray, y_pred: np.ndarray):
    """
    Plota a matriz de confusão do modelo.
    Parameters:
    ----------
        y_test: np.ndarray
            Rótulos reais do conjunto de teste.
        y_pred: np.ndarray
            Rótulos previstos pelo modelo.

    Returns:
    ----------
        None
    """
    cm = confusion_matrix(one_hot_decode(y_test), y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')  
    plt.title('Matriz de Confusão')
    plt.xlabel('Predito')
    plt.ylabel('Real')
    plt.show()

## 

## ⚙️ Função Principal

Coordenação de todas as etapas do projeto, desde o carregamento dos dados até a avaliação do modelo.

In [ ]:
def main(train_lenet=True, train_efficient=True, batch_size=32, epochs=15, use_data_augmentation=True):
    """
    Função principal para classificação de imagens do dataset STL-10.
    Inclui o treinamento do LeNet-5 (modelo modificado) e do EfficientNet
    (versão melhorada com data augmentation embutido e fine-tuning em duas etapas).
    """
    print("1. Carregando e preparando dados...")
    start_time = time.time()
    
    # Carregar rótulos e processar o dataset STL-10
    print("Processando dataset STL-10...")
    path = download_dataset("jessicali9530/stl10")
    train_labels, test_labels = load_train_test_labels(path)
    
    # Carregar imagens
    X_train_raw, y_train = load_images_from_folder(path, train_labels, 'train_images')
    X_test_raw, y_test = load_images_from_folder(path, test_labels, 'test_images')
    
    # Normalização
    X_train = X_train_raw.astype('float32') / 255.0
    X_test = X_test_raw.astype('float32') / 255.0
    
    # Divisão train/validation
    X_train, X_val, y_train_raw, y_val_raw = dataset_split(X_train, y_train)
    
    # Definir número de classes
    num_classes = 10
    
    # One-hot encoding
    y_train_one_hot = one_hot_encode(y_train_raw, num_classes)
    y_val_one_hot = one_hot_encode(y_val_raw, num_classes)
    y_test_one_hot = one_hot_encode(y_test, num_classes)
    
    print(f"Formato dos dados de treino: {X_train.shape}")
    print(f"Formato dos dados de teste: {X_test.shape}")
    print(f"Rótulos de treino: {np.unique(y_train_raw, return_counts=True)}")
    print(f"Rótulos de teste: {np.unique(y_test, return_counts=True)}")
    
    # Dicionários para armazenar modelos e históricos
    models = {'lenet': None, 'efficient': None}
    histories = {'lenet': None, 'efficient': None}
    
    # Treinamento do LeNet-5 (mantendo sua implementação atual)
    if train_lenet:
        print("\n2. Criando e treinando o modelo LeNet-5...")
        lenet_model = lenet_5(input_shape=X_train.shape[1:], num_classes=num_classes)
        lenet_model.summary()
        
        # Utiliza funções de treinamento existentes (com ou sem augmentation)
        if use_data_augmentation:
            lenet_history = model_fit(
                lenet_model,
                X_train,
                y_train_one_hot,
                X_val,
                y_val_one_hot,
                epochs=epochs,
                batch_size=batch_size
            )
        else:
            lenet_history = model_fit_no_augmentation(
                lenet_model,
                X_train,
                y_train_one_hot,
                X_val,
                y_val_one_hot,
                epochs=epochs,
                batch_size=batch_size
            )
        
        models['lenet'] = lenet_model
        histories['lenet'] = lenet_history
    
    # Treinamento do EfficientNet melhorado
    if train_efficient:
        print("\n3. Criando e treinando o modelo EfficientNet (versão melhorada)...")
        efficient_model = create_improved_efficientnet(input_shape=X_train.shape[1:], num_classes=num_classes)
        efficient_model.summary()
        
        # Preparar conjuntos de dados com tf.data para aproveitar o data augmentation embutido
        train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train_one_hot))
        train_dataset = train_dataset.shuffle(1000).batch(batch_size)
        val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val_one_hot)).batch(batch_size)
        
        # Treinamento em duas etapas: inicial e fine-tuning
        efficient_model, efficient_history = fine_tune_efficientnet(
            efficient_model,
            train_data=train_dataset,
            validation_data=val_dataset,
            epochs_initial=20,
            epochs_fine_tune=30
        )
        
        models['efficient'] = efficient_model
        histories['efficient'] = efficient_history
    
    # Avaliação dos modelos
    if train_lenet or train_efficient:
        print("\n4. Avaliando modelos no conjunto de teste...")
        
        results = {
            'lenet': {'accuracy': 0, 'recall': 0, 'precision': 0, 'f1': 0},
            'efficient': {'accuracy': 0, 'recall': 0, 'precision': 0, 'f1': 0}
        }
        
        y_test_classes = np.argmax(y_test_one_hot, axis=1)
        predictions = {}
        
        # Avaliação do LeNet-5
        if train_lenet:
            lenet_scores = evaluate_model(models['lenet'], X_test, y_test_one_hot)
            results['lenet']['accuracy'] = lenet_scores['accuracy']
            
            lenet_preds = models['lenet'].predict(X_test)
            lenet_pred_classes = np.argmax(lenet_preds, axis=1)
            predictions['lenet'] = lenet_pred_classes
            
            results['lenet']['recall'] = recall_score(y_test_classes, lenet_pred_classes, average='macro')
            results['lenet']['precision'] = precision_score(y_test_classes, lenet_pred_classes, average='macro')
            results['lenet']['f1'] = f1_score(y_test_classes, lenet_pred_classes, average='macro')
            
            print("\nMatriz de confusão para LeNet-5:")
            imprime_matriz_confusao(y_test_one_hot, lenet_pred_classes)
        
        # Avaliação do EfficientNet
        if train_efficient:
            efficient_scores = evaluate_model(models['efficient'], X_test, y_test_one_hot)
            results['efficient']['accuracy'] = efficient_scores['accuracy']
            
            efficient_preds = models['efficient'].predict(X_test)
            efficient_pred_classes = np.argmax(efficient_preds, axis=1)
            predictions['efficient'] = efficient_pred_classes
            
            results['efficient']['recall'] = recall_score(y_test_classes, efficient_pred_classes, average='macro')
            results['efficient']['precision'] = precision_score(y_test_classes, efficient_pred_classes, average='macro')
            results['efficient']['f1'] = f1_score(y_test_classes, efficient_pred_classes, average='macro')
            
            print("\nMatriz de confusão para EfficientNet:")
            imprime_matriz_confusao(y_test_one_hot, efficient_pred_classes)
        
        # Impressão dos resultados
        print("\n5. Resultados da avaliação:")
        print("| Modelo      | Acurácia      | Recall        | Precision     | F1 Score      |")
        print("|-------------|---------------|---------------|---------------|---------------|")
        
        if train_lenet:
            r = results['lenet']
            print(f"| LeNet-5     | {r['accuracy']*100:.2f}%      | {r['recall']:.4f}      | {r['precision']:.4f}      | {r['f1']:.4f}      |")
        
        if train_efficient:
            r = results['efficient']
            print(f"| EfficientNet| {r['accuracy']*100:.2f}%      | {r['recall']:.4f}      | {r['precision']:.4f}      | {r['f1']:.4f}      |")
        
        # Visualização dos históricos de treinamento
        if train_lenet and train_efficient:
            print("\n6. Visualizando histórico de treinamento...")
            plt.figure(figsize=(12, 5))
            
            # Plot de perda
            plt.subplot(1, 2, 1)
            plt.plot(histories['lenet'].history['loss'], label='LeNet-5 Treino')
            plt.plot(histories['lenet'].history['val_loss'], label='LeNet-5 Validação')
            # Para EfficientNet, combinamos os históricos inicial e de fine-tuning
            efficient_loss = histories['efficient'][0].history['loss'] + histories['efficient'][1].history['loss']
            efficient_val_loss = histories['efficient'][0].history['val_loss'] + histories['efficient'][1].history['val_loss']
            plt.plot(efficient_loss, label='EfficientNet Treino')
            plt.plot(efficient_val_loss, label='EfficientNet Validação')
            plt.title('Comparação de Perda')
            plt.ylabel('Perda')
            plt.xlabel('Época')
            plt.legend()
            
            # Plot de acurácia
            plt.subplot(1, 2, 2)
            plt.plot(histories['lenet'].history['accuracy'], label='LeNet-5 Treino')
            plt.plot(histories['lenet'].history['val_accuracy'], label='LeNet-5 Validação')
            efficient_acc = histories['efficient'][0].history['accuracy'] + histories['efficient'][1].history['accuracy']
            efficient_val_acc = histories['efficient'][0].history['val_accuracy'] + histories['efficient'][1].history['val_accuracy']
            plt.plot(efficient_acc, label='EfficientNet Treino')
            plt.plot(efficient_val_acc, label='EfficientNet Validação')
            plt.title('Comparação de Acurácia')
            plt.ylabel('Acurácia')
            plt.xlabel('Época')
            plt.legend()
            plt.tight_layout()
            plt.show()
        elif train_lenet:
            imprime_loss_accuracy(histories['lenet'])
        elif train_efficient:
            imprime_loss_accuracy(histories['efficient'])
        
        # Visualização de algumas previsões
        if train_lenet or train_efficient:
            print("\n7. Visualizando algumas previsões...")
            n_samples = 5
            indices = np.random.randint(0, len(X_test), n_samples)
            
            fig, axes = plt.subplots(n_samples, 1 + train_lenet + train_efficient, figsize=(15, 3 * n_samples))
            class_names = ['avião', 'pássaro', 'carro', 'gato', 'veado', 'cachorro', 'cavalo', 'macaco', 'navio', 'caminhão']
            
            for i, idx in enumerate(indices):
                axes[i, 0].imshow(X_test_raw[idx])
                axes[i, 0].set_title(f"Real: {class_names[y_test_classes[idx]]}")
                axes[i, 0].axis('off')
                
                col = 1
                if train_lenet:
                    pred_class = predictions['lenet'][idx]
                    color = 'green' if pred_class == y_test_classes[idx] else 'red'
                    axes[i, col].imshow(X_test_raw[idx])
                    axes[i, col].set_title(f"LeNet: {class_names[pred_class]}", color=color)
                    axes[i, col].axis('off')
                    col += 1
                
                if train_efficient:
                    pred_class = predictions['efficient'][idx]
                    color = 'green' if pred_class == y_test_classes[idx] else 'red'
                    axes[i, col].imshow(X_test_raw[idx])
                    axes[i, col].set_title(f"EfficientNet: {class_names[pred_class]}", color=color)
                    axes[i, col].axis('off')
            
            plt.tight_layout()
            plt.show()
        
        end_time = time.time()
        total_time = end_time - start_time
        print(f"\nTempo total de execução: {total_time/60:.2f} minutos")
    
    return models.get('lenet'), models.get('efficient')

# Execução da função principal
if __name__ == "__main__":
    lenet_model, efficient_model = main(train_lenet=True, train_efficient=True, 
                                        batch_size=32, epochs=15, 
                                        use_data_augmentation=True)

1. Carregando e preparando dados...
Processando dataset STL-10...
Conteúdo do diretório C:\Users\WINN\.cache\kagglehub\datasets\jessicali9530\stl10\versions\3: ['test.json', 'test_images', 'train.json', 'train_images', 'unlabeled_images']
Procurando por imagens no caminho: C:\Users\WINN\.cache\kagglehub\datasets\jessicali9530\stl10\versions\3\train_images
Usando diretório de imagens: C:\Users\WINN\.cache\kagglehub\datasets\jessicali9530\stl10\versions\3\train_images
Exemplo de estrutura de rótulo: {'file': 'train_images/train_image_png_1.png', 'label': 1}
Total de imagens carregadas: 5000
Conteúdo do diretório C:\Users\WINN\.cache\kagglehub\datasets\jessicali9530\stl10\versions\3: ['test.json', 'test_images', 'train.json', 'train_images', 'unlabeled_images']
Procurando por imagens no caminho: C:\Users\WINN\.cache\kagglehub\datasets\jessicali9530\stl10\versions\3\test_images
Usando diretório de imagens: C:\Users\WINN\.cache\kagglehub\datasets\jessicali9530\stl10\versions\3\test_images
E

c:\dev\machine_learning\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_18 (Conv2D)              │ (None, 92, 92, 6)      │           456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_12            │ (None, 46, 46, 6)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 46, 46, 6)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 42, 42, 16)     │         2,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_13            │ (None, 21, 21, 16)     │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_21 (Dropout)            │ (None, 21, 21, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 17, 17, 120)    │        48,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 34680)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 84)             │     2,913,204 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ (None, 84)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 10)             │           850 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,965,046 (11.31 MB)

...

c:\dev\machine_learning\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 829ms/step - accuracy: 0.0896 - loss: 2.3236

125/125 ━━━━━━━━━━━━━━━━━━━━ 114s 861ms/step - accuracy: 0.0896 - loss: 2.3236 - val_accuracy: 0.0970 - val_loss: 2.3527 - learning_rate: 0.0010
Epoch 2/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 90s 713ms/step - accuracy: 0.0939 - loss: 2.3349 - val_accuracy: 0.0930 - val_loss: 2.3275 - learning_rate: 0.0010
Epoch 3/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 737ms/step - accuracy: 0.0869 - loss: 2.3353

125/125 ━━━━━━━━━━━━━━━━━━━━ 147s 756ms/step - accuracy: 0.0870 - loss: 2.3352 - val_accuracy: 0.1130 - val_loss: 2.3376 - learning_rate: 0.0010
Epoch 4/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 97s 772ms/step - accuracy: 0.0927 - loss: 2.3457 - val_accuracy: 0.1030 - val_loss: 2.3396 - learning_rate: 0.0010
Epoch 5/15
  5/125 ━━━━━━━━━━━━━━━━━━━━ 1:39 832ms/step - accuracy: 0.1375 - loss: 2.3240

KeyboardInterrupt: 

## 🎯 Conclusão: Redes Neurais para Classificação de Câncer

### 📊 Avaliação das Arquiteturas de Redes Neurais

Os experimentos com diferentes arquiteturas nos permitiram obter resultados precisos para a classificação de tumores:

| Arquitetura | Acurácia (Teste) | F1-Score | Recall | Precision |
|-------------|------------------|----------|---------|-----------|
| **Deep** | 92.98% | 94.37% | 98.53% | 90.54% |
| **Dropout** | 92.98% | 94.37% | 98.53% | 90.54% |
| **Regularized** | 91.23% | 92.86% | 95.59% | 90.28% |
| **Softmax** | 89.47% | 91.30% | 92.65% | 90.00% |
| **Wide** | 92.11% | 93.62% | 97.06% | 90.41% |

### 🧠 Principais Insights dos Resultados

1. **Performance das Arquiteturas**
   - As arquiteturas Deep e Dropout alcançaram os melhores resultados (92.98% de acurácia)
   - Ambas demonstraram excelente recall (98.53%), crucial para minimizar falsos negativos
   - A precisão se manteve consistente em torno de 90% para todas as arquiteturas

2. **Generalização e Overfitting**
   - Dropout apresentou a maior diferença treino-validação (-10.21%), indicando forte regularização
   - Arquitetura Deep mostrou melhor equilíbrio com diferença treino-validação de -1.35%
   - Softmax foi a mais estável com diferença mínima (0.18%), mas com menor performance geral

3. **Trade-offs Observados**
   - Maior complexidade (Deep) → Melhor performance, mas requer mais cuidado com overfitting
   - Dropout → Excelente regularização, mas com custo de performance no treino (71.37%)
   - Regularização L2 → Boa generalização (-4.22% diferença), mantendo performance aceitável

### 🏥 Implicações Práticas

1. **Escolha da Arquitetura**
   - Para máxima sensibilidade: Deep ou Dropout (98.53% recall)
   - Para maior estabilidade: Regularized (menor variação entre métricas)
   - Para rápida implementação: Wide (boa performance com arquitetura mais simples)

2. **Considerações Clínicas**
   - Alto recall (>95% na maioria) → Poucos falsos negativos
   - Precisão consistente (~90%) → Confiabilidade nas detecções positivas
   - F1-Score >92% → Bom equilíbrio entre precisão e recall

### 🔍 Análise das Arquiteturas

1. **Deep (Melhor Performance)**
   - Acurácia: 92.98%
   - Recall: 98.53% (menor taxa de falsos negativos)
   - F1-Score: 94.37% (bom equilíbrio entre precisão e recall)
   - AUC de 0.831 indica boa capacidade discriminativa

2. **Dropout (Melhor Regularização)**
   - Acurácia: 92.98%
   - Recall: 98.53% (semelhante ao Deep)
   - F1-Score: 94.37%
   - Dropout de 0.5 aplicado para evitar overfitting
   - AUC de 0.831, similar ao Deep, mas com maior diferença treino-validação (-10.21%)
   - Performance de treino mais baixa (71.37%)

3. **Regularized (Melhor Generalização)**
   - Acurácia: 91.23%
   - Recall: 95.59% (menor que Deep e Dropout)
   - F1-Score: 92.86%
   - Regularização L2 aplicada, resultando em menor diferença treino-validação (-4.22%)
   - AUC de 0.831, indicando boa capacidade discriminativa
   - Bom equilíbrio entre performance e generalização

4. **Softmax (Menor Performance)**
   - Acurácia: 89.47%
   - Recall: 92.65% (menor taxa de falsos negativos)
   - F1-Score: 91.30%
   - Diferença treino-validação mínima (0.18%), indicando boa estabilidade
   - AUC de 0.831, mas com menor capacidade discriminativa
   - Performance geral inferior às outras arquiteturas

5. **Wide (Arquitetura Simples)**
   - Acurácia: 92.11%
   - Recall: 97.06% (menor taxa de falsos negativos)
   - F1-Score: 93.62%
   - Diferença treino-validação de -4.22%, indicando boa generalização
   - AUC de 0.831, similar às outras arquiteturas
   - Boa performance com arquitetura mais simples
   - Menor complexidade, mas com trade-off em relação à profundidade da rede

### 🚀 Recomendações para Melhorias

1. **Otimizações Técnicas**
   - Explorar combinações de Dropout com L2 para melhor regularização
   - Implementar validação cruzada para resultados mais robustos
   - Investigar técnicas de ensemble combinando diferentes arquiteturas

2. **Validação Adicional**
   - Testar com datasets externos para confirmar generalização
   - Realizar testes em ambiente clínico controlado
   - Comparar com performance de especialistas humanos

3. **Desenvolvimento Futuro**
   - Implementar visualizações de explicabilidade (SHAP, LIME)
   - Desenvolver interface amigável para uso médico
   - Expandir para classificação multiclasse de subtipos de tumor

Este estudo demonstrou que diferentes arquiteturas de redes neurais podem alcançar alta performance na classificação de tumores, com acurácias superiores a 90% e recall chegando a 98.53%. A escolha da arquitetura deve considerar o equilíbrio entre performance e generalização, com as versões Deep e Dropout mostrando os resultados mais promissores para aplicação prática.